In [1]:
import math
import random
from IPython.display import clear_output

#### WHAT ARE GOOD METRICS FOR EVALUATING A MENU?
***

- How long it takes to find a menu item.
- Order of items.
- Number of items per menu.
- Groups.


- Single vs. Multi-objective problems.
    - Do you want to solve big problems all at once?
    - Or have a recursive approach
        - One big problem = 10 medium problem = 100 small problems.
        

- Whitebox model without the knowledge or experience.


- Is it better to use a multiplier for different objective functions and then sum them together.
- Or a recursive algo.

#### PARAMS

In [2]:
frequencies = {
    'help': 1,
    'save': 1,
    'close': 1,
    'print': 1,
    'open': 1,
    'about': 1,
    '-': 0
}

In [3]:
associations = {
    'open':  { 'open': 1.0, 'save': 0.5, 'close': 0.3 }, 
    'save':  { 'save': 1.0, 'close': 0.2, 'open': 0.5}, 
    'close': { 'close': 1.0, 'open': 0.2, 'save':0.2}, 
    'help':  { 'help': 1.0, 'about': 0.2 }, 
    'about': { 'about':1.0, 'help': 0.1 },
    'print': { 'print': 1.0 }
}

In [4]:
menu = ['help', '-', 'open', 'save', '-', 'print', 'close', '-', 'about']

#### FITTS LAW OBJECTIVE FUNC

In [5]:
def fitts_law(menu, frequencies):
    
    # FITTS LAW PARAMS
    a = 0.2 
    b = 0.3
    
    # COMPUTE THE COST OF EACH MENU ITEM
    item_costs = [a + b * math.log(i + 1) for i in range (0, len(menu))]
    
    # PLACEHOLDER
    total_cost = 0.0
    
    # COMPUTE THE CUMULATIVE MENU COST
    for i in range (0, len(menu)):
        total_cost += item_costs[i] * frequencies[menu[i]]
        
    return total_cost

In [6]:
fitts_law(menu, frequencies)

3.7259402536222765

#### ITEM ASSOCIATION OBJECTIVE FUNC

In [31]:
def item_assoc(menu, associations):
    
    # TOTAL COST
    total_cost = 0.0
    
    # LOOP THROUGH MENU ITEMS, SKIP LINES
    for current in range (0, len(menu)):
        if menu[current] == "-": continue

        # LOOP THROUGH SUCCESSORS, SKIP LINES
        for successor in range (current+1, len(menu)):
            if menu[successor] == "-": continue
            
            # SHORTHANDS
            current_item = menu[current]
            successor_item = menu[successor]
            
            # CONTAINER
            item_cost = 0.0
            
            # FETCH ITEM ASSOC SCORE
            try: item_cost = associations[current_item][successor_item]
            except:
                try: item_cost = associations[successor_item][current_item]
                except: pass
                
            # NORMALIZE THE COST
            normalizer = abs(current - successor)
            item_cost *= normalizer
                
            # ITEMS ARE UNRELATED, ADD LARGE PENALTY
            if item_cost == 0.0 and (normalizer == 1):
                total_cost += 1.0
                
            # OTHERWISE, ADD ITEM COST
            else: 
                total_cost += item_cost
                
    return total_cost

In [32]:
item_assoc(menu, associations)

4.9

#### MENU ITEM SEARCH TIME

In [9]:
def search_time(target, menu, associations):
    
    # THE ACCUMULATED COST
    total_cost = 0.0
    
    # HELPERS
    group_boundary = False
    target_found = False
    anchor = None
    
    # MOVEMENT COSTS
    item_cost = 0.3
    group_cost = 0.4
    
    # LOOP THROUGH MENU ITEMS
    for current in range (0, len(menu)):
        
        # IF A SPLITTER IS FOUND
        if menu[current] == "-": 
            group_boundary = True
            anchor = None
            continue
            
        # IF THE FIRST ELEMENT IS FOUND
        if current == 0:
            group_boundary = True
            
        # POTENTIAL ANCHOR FOUND
        if group_boundary and (menu[current] != "-"):
            
            # ADD ITEM COST
            total_cost += item_cost
            
            # THE CURRENT ITEM & TARGET ARE ASSOC, SET ANCHOR
            if menu[current] in associations.keys() and (menu[target] in associations[menu[current]]):
                anchor = menu[current]
            
            # OTHERWISE, RESET ANCHOR AND ADD GROUP COST
            elif menu[current] in associations.keys() and (menu[target] not in associations[menu[current]]):
                anchor = None
                total_cost += group_cost
            
            # RESET GRP BOUNDARY
            group_boundary = False
        
        # IF THERE IS NO GRP BOUNDARY OR ANCHOR, ADD ITEM COST
        elif group_boundary == False: 
            if anchor != None:
                total_cost += item_cost           
                
        # IF THE TARGET IS FOUND, BREAK LOOP
        if (anchor != None) and (current == target):
            target_found = True
            break
        
        # POTENTIAL ANCHOR BUT ISNT THE TARGET, ADD ITEM COST & RESET ANCHOR
        elif (anchor != None) and (current != target) and (anchor not in associations[menu[current]].keys()):
            anchor = None
            total_cost += item_cost
        
        # OTHERWISE, JUMP TO NEXT LOOP ROUND
        elif (anchor == None):
            continue
    
    # IF THE TARGET WAS NEVER FOUND, ADD FINAL PENALTY COST
    if not target_found:
        total_cost += item_cost * len(menu)
    
    return round(total_cost, 3)

In [25]:
def search_items(menu, associations):
    
    # HELPERS
    container = {}
    total_time = 0.0
    
    # LOOP THROUGH MENU
    for current in range (0, len(menu)):
        
        # ESTIMATE TIME TO FIND ITEM
        duration = search_time(current, menu, associations)
        
        # PUSH TO CONTAINERS
        container[menu[current]] = duration
        total_time += duration

    # COMPUTE THE AVG TIME
    avg = total_time / len(menu)
        
    return container, avg

In [26]:
values, average = search_items(menu, associations)

In [27]:
average

3.1333333333333333

#### MENU OPTIMIZER

In [11]:
def optimizer(iterations, base_menu, frequencies, associations):
    
    # FETCH OBJECTIVE FUCNS
    global item_assoc, fitts_law
    
    # RESULT TRACKERS
    best_value = float('inf')
    best_design = []
    
    # LOOP THROUGH ITERATIONS
    for index in range (0, iterations):
        
        # PICK A RANDOMIZED MENU ORDER
        menu_candidate = random.sample(base_menu, len(base_menu))
        
        # RUN OBJ FUNCS TO COMPUTE OBJ VALUE
        objective_value = 1.0 * item_assoc(menu_candidate, associations)
        objective_value = 0.2 * fitts_law(menu_candidate, frequencies)

        # IF A BETTER RESULT IS FOUND, OVERWRITE OLD RESULT
        if objective_value < best_value: 
            best_value = objective_value
            best_design = menu_candidate
        
        # PRINT PROGRESS
        if index % 100 == 0:
            clear_output(wait=True)
            print(str(index) + '/' + str(iterations))
        elif index +1 == iterations:
            clear_output(wait=True)
            print(str(iterations) + '/' + str(iterations))
    
    return best_value, best_design

In [12]:
value, design = optimizer(10000, menu, frequencies, associations)

10000/10000


In [13]:
value

0.6347550727206062

In [14]:
design

['open', 'close', 'print', 'save', 'help', 'about', '-', '-', '-']